# Dino Game model fitting

## Summary
* PyTorch DNN to play Dino-Game

### Inputs

* 1D vectors of cropped screenshots of dino games

### Outputs

* A label prediction `[0, 1, 2]` for the key to press (or what action the model should take given the pixel values in the image)

### Modeling task
* Given an input image $X$, output a label for the action to be taken by the model (jump, duck, nothing)

### Evaluation metric
* Classification accuracy 
* Cross-entropy loss for training

### Models
* Multinomial logistic/Softmax regression in other notebooks
* Deep Feed-Forward NNet with ReLU activations and a softmax output layer

### To-do
* Well this model seems to predcict well without overfititng in training --- can't really run things locally after the google collab issues and all but hopefully this performs better than LogReg

In [1]:
import random
import time
import pickle
# Pre-processing
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform

# Numerical packages
import torch
import pandas as pd
import numpy as np
import torch.optim as optim

# Plotting & eval metrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

%matplotlib inline

In [2]:
def map_keys(label_vec):
    """Take a vector of key-press labels and convert them to proper encodings. """
    result = np.zeros_like(label_vec)
    key_dict = {-1: 0, 38: 1, 40: 2}
    for i in range(label_vec.shape[0]):
        result[i] = key_dict[label_vec[i]]
    return result

In [3]:
def map_keys_rev(pred_vec):
    """ Take a vector of classifications and return keyboard outputs """
    result = torch.zeros_like(pred_vec)
    key_dict = {0: -1, 1: 38, 2: 40}
    for i in range(label_vec.shape[0]):
        result[i] = key_dict[label_vec[i]]
    return result

In [4]:
images = torch.Tensor(np.load('data/screenshots.npy'))
labels = torch.Tensor(map_keys(np.load('data/command_keys.npy')))

In [5]:
n_obs = len(labels)
n_classes = len(np.unique(labels))
n_pixels = images.shape[1]

In [6]:
print(f'{n_obs} images in {n_classes} categories, with {n_pixels} pixels each.')

15673 images in 3 categories, with 129600 pixels each.


In [7]:
# We want an input vector of 3760x(1450x288)
print(f'Shape of input vector: {images.shape}')
print(f'Shape of targets: {labels.shape} w/ unique values {torch.unique(labels)}')

Shape of input vector: torch.Size([15673, 129600])
Shape of targets: torch.Size([15673]) w/ unique values tensor([0., 1., 2.])


#### Create DataSet class for PyTorch

In [8]:
class DinoImagesDataset(Dataset):
    """ A Dataset class for processing screenshots"""
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        
    def __len__(self):
        """ Give number of total observations in the dataset. """
        return len(self.labels)
    
    def __getitem__(self, idx):
        """ Return a training sample from the dataset. """
        label = self.labels[idx].type(torch.LongTensor)
        image = self.images[idx].type(torch.FloatTensor)
        #sample = {"Image": image, "Label": label}
        return image, label
    
    def num_pixels(self):
        """ Return the number of pixels in an image"""
        return len(self.images[0])

#### Initialize dataset with new dino-game data

In [9]:
dino_data = DinoImagesDataset(images, labels)

In [10]:
print(f'N_obs: {dino_data.__len__()}, with {dino_data.num_pixels()} pixels')

N_obs: 15673, with 129600 pixels


#### Setup train-test splits

In [11]:
train_size = int(0.8 * len(dino_data))
test_size = len(dino_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dino_data, [train_size, test_size])
print(f'{train_size} training images and {test_size} testing images')
dino_train_DL = DataLoader(train_dataset, batch_size=32, shuffle=True)
dino_test_DL = DataLoader(test_dataset, batch_size=test_size, shuffle=False)

12538 training images and 3135 testing images


#### Create PyTorch model

In [12]:
class BasicNNet(torch.nn.Module):
    """ Basic NNet for playing the dino game """
    def __init__(self, input_size):
        super().__init__()
        self.layer1 = torch.nn.Linear(input_size, 500)
        self.activation1 = torch.nn.ReLU()
        self.layer2 = torch.nn.Linear(500, 250)
        self.activation2 = torch.nn.ReLU()
        self.layer3 = torch.nn.Linear(250, 25)
        self.activation3 = torch.nn.ReLU()
        self.layer4 = torch.nn.Linear(25, 3)
        
    def forward(self, x):
        """ Forward pass on images to calculate log-probability of each key press given image pixels"""
        x = self.layer1(x)
        x = self.activation1(x)
        x = self.layer2(x)
        x = self.activation2(x)
        x = self.layer3(x)
        x = self.activation3(x)
        x = self.layer4(x)
        
        log_probs = torch.nn.functional.log_softmax(x, dim=1)
        
        return log_probs

#### Instantiate and train model

In [13]:
model = BasicNNet(n_pixels)
print(model)

BasicNNet(
  (layer1): Linear(in_features=129600, out_features=500, bias=True)
  (activation1): ReLU()
  (layer2): Linear(in_features=500, out_features=250, bias=True)
  (activation2): ReLU()
  (layer3): Linear(in_features=250, out_features=25, bias=True)
  (activation3): ReLU()
  (layer4): Linear(in_features=25, out_features=3, bias=True)
)


In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

n_epochs = 10

In [15]:
model.train()
for epoch in range(n_epochs):
    
    running_loss = 0.0 # loss for current epoch
    for i, data in enumerate(dino_train_DL):
        # Get inputs and labels
        inputs, labels = data
        
        # Zero gradient (this is common practice with pytorch on each batch
        optimizer.zero_grad()
        
        # Now perform forward pass
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        
        # Calculate gradients of loss function 
        loss.backward()
        
        # Backprop step to update params
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 1.122
[2,   200] loss: 0.720
[3,   200] loss: 0.695
[4,   200] loss: 0.700
[5,   200] loss: 0.686
[6,   200] loss: 0.697
[7,   200] loss: 0.703
[8,   200] loss: 0.708
[9,   200] loss: 0.695
[10,   200] loss: 0.708
Finished Training


#### Evaluate models on out-of-sample (OOS) screenshots

In [16]:
# Function to test the model with the test dataset and print the accuracy for the test images
def test_accuracy(model):
    """ Test accuracy of model """
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in dino_test_DL:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

In [17]:
test_accuracy(model)

78.59649122807018

#### Save models

In [18]:
# torch.save(model, 'models/DNN.model')
torch.save(model, 'models/nn.model')